# Selective flow constraints with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

The [Operations Problems example]](../../notebook/3_PowerSimulations_examples/sequential_simulations.ipynb)
shows the basic building blocks of building optimization problems with PowerSimulations.jl.
This example shows how to customize the enforcement of branch flow constraints as is common
when trying to build large scale simulations.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [3]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Create a `System` from RTS-GMLC data
We can just use the
[parsing tabular data example.](../../notebook/2_PowerSystems_examples/04_parse_tabulardata.jl)

In [4]:
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "2_PowerSystems_examples", "04_parse_tabulardata.jl"))

┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:125
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pow

┌ Info: Adding contributing generators for Spin_Up_R2 by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:675
┌ Warning: User-defined column name Startup Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Shutdown Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Status at Start is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Time at Status is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Start Cost Cold is not in dat

┌ Info: Adding contributing generators for Reg_Down by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:675
┌ Warning: User-defined column name Startup Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Shutdown Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Status at Start is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Time at Status is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Start Cost Cold is not in dataf

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,109
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,73
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
5,HVDCLine,DCBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
7,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,19
8,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,105
9,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3


### Selecting flow limited lines
Since PowerSimulations will apply constraints by component type (e.g. Line), we need to
change the component type of the lines on which we want to enforce flow limits. So, let's
change the device type of certain branches from Line to MonitoredLine differentiate
treatment when we build the model. Here, we can select inter-regional lines, or lines
above a voltage threshold.

In [5]:
for line in get_components(Line, sys)
    if (get_base_voltage(get_from(get_arc(line))) >= 230.0) &&
       (get_base_voltage(get_to(get_arc(line))) >= 230.0)
        #if get_area(get_from(get_arc(line))) != get_area(get_to(get_arc(line)))
        @info "Changing $(get_name(line)) to MonitoredLine"
        convert_component!(MonitoredLine, line, sys)
    end
end

┌ Info: Changing C31-2 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing B26 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing A32-2 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing CA-1 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing B34 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing B19 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing B31-2 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing C30 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing B27 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing A21 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing A32-1 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing B29 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing A18 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing C24 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing C28 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing A29 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing AB2 to MonitoredLine
└ @ Main In[5]:5
┌ Info: Changing C27 to MonitoredLine
└ @ Main In[5]:5
┌

## Build an `OperationsProblem`

In [6]:
uc_prob =
    UnitCommitmentProblem(sys, optimizer = solver, horizon = 24, network = DCPPowerModel)

┌ Info: Unit System changed to InfrastructureSystems.UnitSystemModule.UnitSystem.SYSTEM_BASE = 0
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/base.jl:289
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: No variable cost defined for 201_HYDRO_4
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/devices_models/devices/common/cost_functions.jl:346
┌ Warning: The data doesn't include devices of type Transformer2W, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/devices_models/device_constructors/common/constructor_validations.jl:3



Operations Problem Specification

  transmission:  DCPPowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = HydroDispatchRunOfRiver
      Generators:
        device_type = ThermalStandard
        formulation = ThermalBasicUnitCommitment
      DistRE:
        device_type = RenewableFix
        formulation = FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
    

Let's change the formulation of the `Line` components to an unbounded flow formulation.
This formulation still enforces Kirchoff's laws, but does not apply flow constraints.

In [7]:
set_branch_model!(uc_prob, :L, DeviceModel(Line, StaticLineUnbounded))

┌ Info: Unit System changed to InfrastructureSystems.UnitSystemModule.UnitSystem.SYSTEM_BASE = 0
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/base.jl:289
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: No variable cost defined for 201_HYDRO_4
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/devices_models/devices/common/cost_functions.jl:346
┌ Warning: The data doesn't include devices of type Transformer2W, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/devices_models/device_constructors/common/constructor_validations.jl:3


Notice that there is no entry for `MonitoredLine` branches. So, let's add one.

In [8]:
construct_device!(uc_prob, :ML, DeviceModel(MonitoredLine, StaticLine))

Solve the relaxed problem

In [9]:
solve!(uc_prob)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 1.06275e+06 - 0.78 seconds
Cgl0004I processed model has 13008 rows, 26297 columns (5314 integer (5314 of which binary)) and 61873 elements
Cbc0045I Trying just fixing integer variables (and fixingish SOS).
Cbc0045I Warning: mipstart values could not be used to build a solution.
Cbc0012I Integer solution of 1087676.9 found by DiveCoefficient after 0 iterations and 0 nodes (4.45 seconds)
Cbc0011I Exiting as integer gap of 24929.285 less than 1e-10 or 50%
Cbc0001I Search completed - best objective 1087676.906685174, took 0 iterations and 0 nodes (4.47 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 1.06275e+06 to 1.06275e+06
Probing was tried 0 times and created 0 cuts of which 0 were active after add

PowerSimulations.OperationsProblemResults(100.0, Dict{Symbol,DataFrames.DataFrame}(:P__ThermalStandard => 24×76 DataFrame
 Row │ 322_CT_6  321_CC_1  202_STEAM_3  315_STEAM_1  223_CT_4  223_CT_6  313_C ⋯
     │ Float64   Float64   Float64      Float64      Float64   Float64   Float ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      0.0   0.0         0.3               0.0       0.0       0.0   0.0  ⋯
   2 │      0.0   0.0         0.3               0.0       0.0       0.0   0.0
   3 │      0.0   0.0         0.3               0.0       0.0       0.0   0.0
   4 │      0.0   0.0         0.3               0.0       0.0       0.0   0.0
   5 │      0.0   0.0         0.3               0.0       0.0       0.0   0.0  ⋯
   6 │      0.0   0.0         0.3               0.0       0.0       0.0   0.0
   7 │      0.0   0.0         0.606667          0.0       0.0       0.0   0.0
   8 │      0.0   0.0         0.3               0.0       0.0       0.0   0.0
   9 

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*